Tee yleiset muuttujanimet

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from pyproj import CRS
from shapely import geometry, ops

In [2]:
network2 = gpd.read_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/hki_liikennemaarat_2019_linkkikartta_3067_geomfix.shp") 
#network2 = gpd.read_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/test/Kehä1.shp")

In [3]:
collist = list(network2.columns)

In [4]:
network = gpd.GeoDataFrame(network2, columns=collist)

In [5]:
network["KATULUOKKA"] = network["KL"]

In [6]:
grouped = network.groupby("KATU")

In [7]:
#alternative katulist
katulist = list(network2["KATU"])
my_set = set(katulist)
uniquekadut = list(my_set)

In [8]:
len(uniquekadut)

463

In [9]:
%%capture

testilista = []
homogeeniset_linkit = gpd.GeoDataFrame()
mergetest = gpd.GeoDataFrame()

#for i in katulist: #Iterate over streets
for i in uniquekadut:
    grp = grouped.get_group(i)
    KLgrouped = grp.groupby("KL") #Group by road class
    
    #Iterate over classes in street
    for key, y in KLgrouped:
        KLgrp = KLgrouped.get_group(key)
        #Dissolve road sections that are of the same class in the same street
        dissolved = KLgrp.dissolve(by="KL") 
        dissolved = dissolved.reset_index()
        
        #Select sections which are not multilinestring. They can be added to main GDF.
        if str(type(dissolved.iloc[0, 1])) != "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
            homogeeniset_linkit = homogeeniset_linkit.append(dissolved.iloc[0])
        
        #Select sections constisting of multilinestrings. These sections are not next to each other, and must be made separate for the main GDF
        elif str(type(dissolved.iloc[0, 1])) == "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
            merge = ops.linemerge(dissolved.iloc[0, 1]) #now have the n. of continuous pieces as multilinestring object. Must be merged to join otherwise separate but connected lines.
            
            #Select sections merged to multilinestrings
            if str(type(merge)) == "<class 'shapely.geometry.multilinestring.MultiLineString'>": 
                dissolved3 = gpd.GeoDataFrame() #Create GDF to which we will collect these as seperate sections
                bufdf = gpd.GeoDataFrame() #Create GDF to which we will collect buffers of the seperate sections
                buflist = []
                for b in range(0, (len(merge))): #Create enough rows to accommodate all sections
                    dissolved3 = dissolved3.append(dissolved.iloc[0])
                
                #Replace old geometry with seperate sections
                for x in range(0, len(dissolved3)): 
                    dissolved3["geometry"].iloc[x] = merge[x]
                    
                dissolved3 = dissolved3.reset_index()
 
                #BUFFERS --------- Buffers must be created around the seperate sections selected above. 
                #This is due to links in Helsinki network not always being spatially connected in the shape file (difference of 0.1-2m usually, 
                #even though they represent roads connected in reality. Buffers are used to bridge these links.
                
                #Create buffers for all seperated links
                for z in range(0, len(dissolved3)): 
                    buf = dissolved3["geometry"].iloc[z].buffer(8) #<---------ADJUST BUFFER SIZE HERE-------------
                    buflist.append(buf) #Add buffers to a list
                
                dissolved3 = dissolved3.drop(["index"], axis="columns") #Remove index column from dissolved3
                bufdf["geometry"] = gpd.GeoSeries(buflist) #Add contents of buflist to bufdf 
                bufdf.reset_index()
                
                #Create empty geodataframes to be filled with ID concerning whether buffers touch, and how they touch other buffers
                touchingbufs = gpd.GeoDataFrame()
                maxidissolve = gpd.GeoDataFrame()
                
                touchingbufslist = [] #Here we will collect the information concerning what buffers a buffer touches
                excesslist = [] 
                
                #Iterate over buffers. Add buffers to "touchingbufs" and a corresponding ID-value to touchingbufslist.
                for v in list(range(0, len(bufdf)-1)):
                    
                    # FIRST, check if first feature touches another buffer or stands alone. Append touchingbufslist accordingly
                    if len(touchingbufslist) == 0:
                        if (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])): #Collect intersecting buffers into one df
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("s") #Create ID to identify continuous section
                        elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"]) == False):
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("own") #Append ID
                    #front and back        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])) and (bufdf.loc[v]["geometry"].intersects(bufdf.loc[int(v-1)]["geometry"])): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("fb") #Append ID
                    #back only        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v-1]["geometry"])) and ((bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])) == False): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("b") #Append ID
                    #front only        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v-1]["geometry"]) == False) and ((bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"]))): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("s") #Append ID
                    #unconnected        
                    elif (bufdf.loc[v]["geometry"].intersects(bufdf.loc[v-1]["geometry"]) == False) and ((bufdf.loc[v]["geometry"].intersects(bufdf.loc[v+1]["geometry"])) == False): 
                            touchingbufs = touchingbufs.append(bufdf.loc[v])
                            touchingbufslist.append("own") #Append ID
                    #excess        
                    elif KeyError:
                        excesslist.append(1)


                touchingbufs = touchingbufs.reset_index()
                touchingbufs["startstop"] = gpd.GeoSeries(touchingbufslist) #Add intersecting buffer ID to GDF which contains the buffers
                
                #Give running ID-Values to buffers. These will be used to dissolve continuous sections.
                slist = []
                idlist = []
                for k in range(0, len(touchingbufs)):
                    if touchingbufs["startstop"].iloc[k] == "s":
                        slist.append(1)
                        idlist.append(sum(slist))
                    elif touchingbufs["startstop"].iloc[k] == "fb":
                        idlist.append(sum(slist))
                    elif touchingbufs["startstop"].iloc[k] == "b":
                        idlist.append(sum(slist))
                        #slist.append(1)
                    elif touchingbufs["startstop"].iloc[k] == "own":
                        slist.append(1)
                        idlist.append(sum(slist))
                        test = sum(slist)
                
                if len(touchingbufs) > 0:
                    touchingbufs["dissolve"] = gpd.GeoDataFrame(idlist)
                    touchingbufs["BufID"] = gpd.GeoDataFrame(idlist)
                    touchingbufs["BufID2"] = gpd.GeoDataFrame(idlist)
                    
                    tdissolve = touchingbufs.dissolve(by="dissolve") #Dissolve buffers that touch by the intersecting buffer ID
                if len(tdissolve) > 0: #If tdissolve contains something, add it to maxidissolve GDF
                    maxidissolve = maxidissolve.append(tdissolve)
                               
                # Join with previously merged sections
                bufjoin = gpd.sjoin(dissolved3, maxidissolve, how="left", op="within") #Spatially join links (lines) with buffers
                bufjoin = bufjoin.reset_index()
                bufjoinfinal = bufjoin.dissolve(by="BufID") #Dissolve by BufferID
                bufjoinfinal = bufjoinfinal.reset_index()
                

                for n in range(0, len(bufjoin)): #Add links that were not within buffers
                    if pd.isna(bufjoin["BufID2"].loc[n]):
                        bufjoinfinal = bufjoinfinal.append(bufjoin.iloc[n])
                
                for q in range(0, len(bufjoinfinal)):
                    homogeeniset_linkit = homogeeniset_linkit.append(bufjoinfinal.iloc[q]) # append final links to main GDF

            else: # Any other sections need no attention and can be passed to the main GDF
                dissolved4 = gpd.GeoDataFrame()
                dissolved4 = dissolved4.append(dissolved.iloc[0])
                dissolved4["geometry"].iloc[0] = merge
                homogeeniset_linkit = homogeeniset_linkit.append(dissolved4.iloc[0])
        

homogeeniset_linkit = homogeeniset_linkit.drop(["level_0"], axis="columns")
homogeeniset_linkit = homogeeniset_linkit.reset_index()
homogeeniset_linkit.crs = CRS.from_epsg(3067)

In [10]:
#Drop unnecessary columns
homogeeniset_linkit = homogeeniset_linkit.drop(["HA", "PA", "KA", "RA", "LA", "MP", "RV", "AUTOT", "pituus", "ID_NEW", "index", "index_right", "SY KAVL+RV"], axis="columns")

Calculating KVL for the continuous links:

In [11]:
# Create Link identifier
homogeeniset_linkit["H_ID"] = list(range(0, len(homogeeniset_linkit)))

In [12]:
#Spatial join between the continuous links we just created and the original link network
join = gpd.sjoin(homogeeniset_linkit, network2, how="left", op="contains")

In [13]:
#Calculates mean KVL for continuous links
finaldissolve1 = join.dissolve(by='H_ID', aggfunc='mean')

In [ ]:
finaldissolve1.crs = CRS.from_epsg(3067)

In [14]:
len(finaldissolve1)

671

A small number of links, for whatever reason, are not joined with the "contains" method. So we do the following to obtan KVL for these:

In [15]:
katulist = list(join["KATU_left"])

H_ID = list(join["H_ID"])

katuIDdf = gpd.GeoDataFrame()
katuIDdf["KATU"] = gpd.GeoSeries(katulist)
katuIDdf["H_ID"] = gpd.GeoSeries(H_ID)

C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:6: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  
C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:7: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  import sys


In [16]:
# Create an empty geodataframe to which we will add rows from finaldissolve1 that do not have KVL data
NADF = gpd.GeoDataFrame()
for x in range(0, len(finaldissolve1)):
    if pd.isna(finaldissolve1["SY KAVL+RV"].iloc[x]):
        row = finaldissolve1.iloc[x]
        NADF = NADF.append(finaldissolve1.iloc[x])

# Create Link identifier
NADF["H_ID"] = list(range(0, len(NADF))) 
NADF.crs = CRS.from_epsg(3067)

#Create list containing street name and KL data in the correct order. Street names and KL must be re-added as they do not "survive" the dissolution made earlier.
NADFIND = list(NADF.index)
KL_LIST = list(NADF["KATULUOKKA"])
katulist2 = []
for a in range(0, len(NADFIND)):
    for b in range(0, len(katuIDdf)):
        if int(NADFIND[a]) == int(katuIDdf["H_ID"].iloc[b]):
            katulist2.append(str(katuIDdf["KATU"].iloc[b]))
            
NADF = NADF.drop(["index_right"], axis="columns")

In [17]:
NADFIND_FINALDISSOLVE1 = list(finaldissolve1.index)
katulist3 = []

for c in range(0, len(NADFIND_FINALDISSOLVE1)):
    for d in range(0, len(homogeeniset_linkit)):
        if int(NADFIND_FINALDISSOLVE1[c]) == int(homogeeniset_linkit["H_ID"].iloc[d]):
            katulist3.append(str(homogeeniset_linkit["KATU"].iloc[d]))

In [18]:
finaldissolve1 = finaldissolve1.dropna(subset=['SY KAVL+RV'], axis='rows')

In [19]:
# Add street names to finaldissolve1
finaldissolve1["KATU"] = gpd.GeoDataFrame(katulist3)

In [20]:
len(NADF)

38

In [21]:
NADF.head()

,AUTOT,BufID,BufID2,HA,ID_NEW,KA,KATULUOKKA,KL_left,KL_right,LA,...,PA,RA,RV,SY KAVL+RV,geometry,level_0,linkki_left,linkki_right,pituus,H_ID
20,NaN,1.0,1.0,NaN,NaN,NaN,3.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,MULTILINESTRING Z ((393438.149 6678276.082 0.0...,0.0,2656.0,NaN,NaN,0
21,NaN,NaN,NaN,NaN,NaN,NaN,3.0,3.0,NaN,NaN,...,NaN,NaN,NaN,NaN,"LINESTRING Z (393526.397 6678111.065 0.000, 39...",3.0,2656.0,NaN,NaN,1
35,NaN,14.0,14.0,NaN,NaN,NaN,6.0,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,MULTILINESTRING Z ((386375.291 6680364.899 0.0...,13.0,747.0,NaN,NaN,2
37,NaN,16.0,16.0,NaN,NaN,NaN,6.0,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,MULTILINESTRING Z ((386372.685 6680342.888 0.0...,15.0,747.0,NaN,NaN,3
38,NaN,17.0,17.0,NaN,NaN,NaN,6.0,6.0,NaN,NaN,...,NaN,NaN,NaN,NaN,MULTILINESTRING Z ((386428.785 6680379.524 0.0...,16.0,747.0,NaN,NaN,4


In [22]:
#Find KVL information for links where joining failed. Some may still be unconnected. Currently unsure how to fix, but the number of these links appears very small. 9.6.2020
nulljoin = gpd.sjoin(NADF, network2, how="left", op="within")

In [23]:
len(nulljoin)

38

In [24]:
nulldissolve = nulljoin.dissolve(by='H_ID', aggfunc='mean')

In [25]:
nulldissolve = nulldissolve.drop([
 'AUTOT_left',
 'HA_left',
 'ID_NEW_left',
 'KA_left',
 'KATULUOKKA',
 'KL_left',
 'KL_right',
 'LA_left',
 'LVUOSI_left',
 'LVUOSI_right',
 'MP_left',
 'PA_left',
 'RA_left',
 'RV_left',
 'SY KAVL+RV_left',
 'linkki_left',
 'linkki_right',
 'pituus_left',], axis="columns")

In [26]:
nulldissolve["KATULUOKKA"] = gpd.GeoSeries(KL_LIST)
nulldissolve["KATU"] = gpd.GeoSeries(katulist2)

C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  """Entry point for launching an IPython kernel.
C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:2: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  


In [27]:
finaldissolve1 = finaldissolve1[['geometry', 'KATU', 'KATULUOKKA', 'HA', 'PA', 'KA', 'RA', 'LA', 'MP', 'RV', 'AUTOT', 'SY KAVL+RV']]

In [28]:
nulldissolve = nulldissolve[['geometry', 'HA_right', 'PA_right', 'KA_right', 'RA_right', 'LA_right', 'MP_right', 'RV_right', 'AUTOT_right', 'SY KAVL+RV_right', 'KATU', 'KATULUOKKA']]

In [29]:
new_names = {'HA_right': 'HA', 'PA_right': 'PA', 'KA_right': 'KA', 'RA_right': 'RA', 'LA_right': 'LA',
       'MP_right': 'MP', 'RV_right': 'RV', 'AUTOT_right': 'AUTOT', 'SY KAVL+RV_right': 'SY KAVL+RV'}

nulldissolve = nulldissolve.rename(columns=new_names)

In [30]:
nulldissolve.columns

Index(['geometry', 'HA', 'PA', 'KA', 'RA', 'LA', 'MP', 'RV', 'AUTOT',
       'SY KAVL+RV', 'KATU', 'KATULUOKKA'],
      dtype='object')

In [32]:
finalproduct = finaldissolve1
finalproduct = finalproduct.append(nulldissolve)
finalproduct = finalproduct.reset_index()

In [49]:
finalproduct["ID"] = gpd.GeoSeries(list(range(0, len(finalproduct))))
finalproduct = finalproduct.drop(["H_ID"], axis="columns")

C:\Users\jmjohan\.conda\envs\gis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning:     You are passing non-geometry data to the GeoSeries constructor. Currently,
    it falls back to returning a pandas Series. But in the future, we will start
    to raise a TypeError instead.
  """Entry point for launching an IPython kernel.


In [50]:
finalproduct["pituus"] = finalproduct.geometry.length

In [51]:
finalproduct.crs = CRS.from_epsg(3067)

In [52]:
homogeeniset_linkit.crs = CRS.from_epsg(3067)

In [53]:
finaldissolve1.crs = CRS.from_epsg(3067)

In [54]:
nulldissolve.crs = CRS.from_epsg(3067)

In [55]:
finalproduct.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finalproduct4.shp")

In [40]:
#finalproduct.to_csv("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finalproduct.csv")

In [41]:
#nulldissolve.to_csv("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/nulldissolve.csv")

In [42]:
#finaldissolve1.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finaldissolve.shp")

In [43]:
#finaldissolve1.to_csv("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/finaldissolve.csv")

In [44]:
#nulldissolve.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/nulldissolve2.shp")

In [45]:
#homogeeniset_linkit.to_file("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/homogeeniset_linkit_mergetest_connectionsfixed.shp")

In [46]:
#homogeeniset_linkit.to_csv("C:/data/L3Pilot/Katuverkko/Helsinki/hki_link_liikennemääräkartta2019/homogeeniset_linkit_mergetest_connectionsfixed.csv")